# Using custom containers with AI Platform Training

**Learning Objectives:**
1. Learn how to create a train and a validation split with BigQuery
1. Learn how to wrap a machine learning model into a Docker container and train in on AI Platform
1. Learn how to use the hyperparameter tunning engine on Google Cloud to find the best hyperparameters
1. Learn how to deploy a trained machine learning model Google Cloud as a rest API and query it

In this lab, you develop a  multi-class classification model, package the model as a docker image, and run on **AI Platform Training** as a training application. The training application trains a multi-class classification model that predicts the type of forest cover from cartographic data. The [dataset](../../../datasets/covertype/README.md) used in the lab is based on **Covertype Data Set** from UCI Machine Learning Repository.

Scikit-learn is one of the most useful libraries for machine learning in Python. The training code uses `scikit-learn` for data pre-processing and modeling. 

The code is instrumented using the `hypertune` package so it can be used with **AI Platform** hyperparameter tuning job in searching for the best combination of hyperparameter values by optimizing the metrics you specified.


In [48]:
import json
import os
import numpy as np
import pandas as pd
import pickle
import uuid
import time
import tempfile

from googleapiclient import discovery
from googleapiclient import errors

from google.cloud import bigquery
from jinja2 import Template
from kfp.components import func_to_container_op
from typing import NamedTuple

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

Run the command in the cell below to install gcsfs package.

In [49]:
%pip install gcsfs==0.8

Note: you may need to restart the kernel to use updated packages.


## Prepare lab dataset

Set environment variable so that we can use them throughout the entire lab.

The pipeline ingests data from BigQuery. The cell below uploads the Covertype dataset to BigQuery.


In [50]:
# PROJECT_ID = !(gcloud config get-value core/project)
# PROJECT_ID = PROJECT_ID[0]
# DATASET_ID='covertype_dataset'
# DATASET_LOCATION='US'
# TABLE_ID='covertype'
# DATA_SOURCE='gs://assignment-covertype/small/dataset.csv'
# SCHEMA='Elevation:INTEGER,Aspect:INTEGER,Slope:INTEGER,Horizontal_Distance_To_Hydrology:INTEGER,Vertical_Distance_To_Hydrology:INTEGER,Horizontal_Distance_To_Roadways:INTEGER,Hillshade_9am:INTEGER,Hillshade_Noon:INTEGER,Hillshade_3pm:INTEGER,Horizontal_Distance_To_Fire_Points:INTEGER,Wilderness_Area:STRING,Soil_Type:STRING,Cover_Type:INTEGER'


Next, create the BigQuery dataset and upload the Covertype csv data into a table.


In [51]:
# !bq --location=$DATASET_LOCATION --project_id=$PROJECT_ID mk --dataset $DATASET_ID

In [52]:
# !bq --project_id=$PROJECT_ID --dataset_id=$DATASET_ID load \
# --source_format=CSV \
# --skip_leading_rows=1 \
# --replace \
# $TABLE_ID \
# $DATA_SOURCE \
# $SCHEMA


## Configure environment settings

Set location paths, connections strings, and other environment settings. Make sure to update   `REGION`, and `ARTIFACT_STORE`  with the settings reflecting your lab environment. 

- `REGION` - the compute region for AI Platform Training and Prediction
- `ARTIFACT_STORE` - the Cloud Storage bucket created during installation of AI Platform Pipelines. The bucket name starts with the `qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default` prefix.

 Run gsutil ls without URLs to list all of the Cloud Storage buckets under your default project ID. 

In [53]:
!gsutil ls

gs://artifacts.assignment-371420.appspot.com/
gs://assignment-371420-kubeflowpipelines-default/
gs://assignment-371420_cloudbuild/
gs://assignment-covertype/


**HINT:** For ARTIFACT_STORE, copy the bucket name which starts with the qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default prefix from the previous cell output. 

Your copied value should look like 'gs://qwiklabs-gcp-xx-xxxxxxx-kubeflowpipelines-default').

In [54]:
REGION = 'us-central1'
ARTIFACT_STORE = 'gs://artifacts.assignment-371420.appspot.com' # TO DO: REPLACE WITH YOUR ARTIFACT_STORE NAME

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
DATA_ROOT='{}/data'.format(ARTIFACT_STORE)
JOB_DIR_ROOT='{}/jobs'.format(ARTIFACT_STORE)
TRAINING_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'training', 'dataset.csv')
VALIDATION_FILE_PATH='{}/{}/{}'.format(DATA_ROOT, 'validation', 'dataset.csv')

## Explore the Covertype dataset 

Run the query statement below to scan covertype_dataset.covertype table in BigQuery and return the computed result rows.

In [55]:
%%bigquery
SELECT *
FROM `covertype_dataset.covertype`

Query is running:   0%|          |

Downloading:   0%|          |

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2146,256,34,150,62,1253,122,237,239,511,Cache,C2702,2
1,2582,256,19,400,121,2322,173,248,212,402,Commanche,C2703,2
2,2510,256,27,182,113,335,146,244,229,323,Commanche,C2705,2
3,2442,256,33,120,52,458,127,238,238,443,Commanche,C2705,2
4,2823,256,22,360,42,1530,162,247,219,1415,Commanche,C2705,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3079,255,12,85,14,5527,192,247,193,2423,Rawah,C7202,1
99996,2745,255,12,60,11,1486,192,247,193,2440,Rawah,C7745,4
99997,2999,255,12,342,-29,2350,192,247,193,1310,Commanche,C7755,1
99998,3153,255,12,404,116,2139,192,247,193,994,Commanche,C7756,1


## Create training and validation splits

Use BigQuery to sample training and validation splits and save them to Cloud Storage.
### Create a training split

Run the query below in order to have repeatable sampling of the data in BigQuery. Note that `FARM_FINGERPRINT()` is used on the field that you are going to split your data. It creates a training split that takes 80% of the data using the `bq` command and exports this split into the BigQuery table of `covertype_dataset.training`.

In [56]:
!bq query \
-n 0 \
--destination_table covertype_dataset.training \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (1, 2, 3, 4)' 

Waiting on bqjob_r5c8acd1d6c17a668_000001850d971269_1 ... (2s) Current status: DONE   


Use the bq extract command to export the BigQuery training table to GCS at $TRAINING_FILE_PATH.

In [57]:
!bq extract \
--destination_format CSV \
covertype_dataset.training \
$TRAINING_FILE_PATH

Waiting on bqjob_r47ff6c6a17f693c9_000001850d972b74_1 ... (0s) Current status: DONE   


### Create a validation split

Run the query below to create a validation split that takes 10% of the data using the `bq` command and export this split into the BigQuery table `covertype_dataset.validation`.

In the second cell, use the `bq` command to export that BigQuery validation table to GCS at `$VALIDATION_FILE_PATH.`

In [58]:
!bq query \
-n 0 \
--destination_table covertype_dataset.validation \
--replace \
--use_legacy_sql=false \
'SELECT * \
FROM `covertype_dataset.covertype` AS cover \
WHERE \
MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), 10) IN (8)' 

Waiting on bqjob_r46f73c5c6cbf0a16_000001850d973ba8_1 ... (1s) Current status: DONE   


In [59]:
!bq extract \
--destination_format CSV \
covertype_dataset.validation \
$VALIDATION_FILE_PATH

Waiting on bqjob_r5b949a58b1e12fb0_000001850d975051_1 ... (0s) Current status: DONE   


In [60]:
TRAINING_FILE_PATH

'gs://artifacts.assignment-371420.appspot.com/data/training/dataset.csv'

In [61]:
df_train = pd.read_csv(TRAINING_FILE_PATH)
df_validation = pd.read_csv(VALIDATION_FILE_PATH)
print(df_train.shape)
print(df_validation.shape)

(39919, 13)
(10023, 13)


## Develop a training application

### Configure the `sklearn` training pipeline.

The training pipeline preprocesses data by standardizing all numeric features using `sklearn.preprocessing.StandardScaler` and encoding all categorical features using `sklearn.preprocessing.OneHotEncoder`. It uses stochastic gradient descent linear classifier (`SGDClassifier`) for modeling.

In [62]:
numeric_feature_indexes = slice(0, 10)
categorical_feature_indexes = slice(10, 12)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', SGDClassifier(loss='log', tol=1e-3))
])

### Convert all numeric features to `float64`

To avoid warning messages from `StandardScaler` all numeric features are converted to `float64`.

In [63]:
num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}

df_train = df_train.astype(num_features_type_map)
df_validation = df_validation.astype(num_features_type_map)

### Run the pipeline locally.

In [64]:
X_train = df_train.drop('Cover_Type', axis=1)
y_train = df_train['Cover_Type']
X_validation = df_validation.drop('Cover_Type', axis=1)
y_validation = df_validation['Cover_Type']

pipeline.set_params(classifier__alpha=0.001, classifier__max_iter=200)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  slice(0, 10, None)),
                                                 ('cat', OneHotEncoder(),
                                                  slice(10, 12, None))])),
                ('classifier',
                 SGDClassifier(alpha=0.001, loss='log', max_iter=200))])

### Calculate the trained model's accuracy.

In [65]:
accuracy = pipeline.score(X_validation, y_validation)
print(accuracy)

0.6930060860021949


### Prepare the hyperparameter tuning application.
Since the training run on this dataset is computationally expensive you can benefit from running a distributed hyperparameter tuning job on AI Platform Training.

In [66]:
TRAINING_APP_FOLDER = 'training_app'
os.makedirs(TRAINING_APP_FOLDER, exist_ok=True)

### Write the tuning script. 

Notice the use of the `hypertune` package to report the `accuracy` optimization metric to AI Platform hyperparameter tuning service.

In [67]:
%%writefile {TRAINING_APP_FOLDER}/train.py

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import subprocess
import sys

import fire
import pickle
import numpy as np
import pandas as pd

import hypertune

from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder


def train_evaluate(job_dir, training_dataset_path, validation_dataset_path, alpha, max_iter, hptune):
    
    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    numeric_feature_indexes = slice(0, 10)
    categorical_feature_indexes = slice(10, 12)

    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_feature_indexes),
        ('cat', OneHotEncoder(), categorical_feature_indexes) 
    ])

    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', SGDClassifier(loss='log',tol=1e-3))
    ])

    num_features_type_map = {feature: 'float64' for feature in df_train.columns[numeric_feature_indexes]}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map) 

    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    X_train = df_train.drop('Cover_Type', axis=1)
    y_train = df_train['Cover_Type']

    pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df_validation.drop('Cover_Type', axis=1)
        y_validation = df_validation['Cover_Type']
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
          hyperparameter_metric_tag='accuracy',
          metric_value=accuracy
        )

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = "{}/{}".format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path], stderr=sys.stdout)
        print("Saved model in: {}".format(gcs_model_path)) 
    
if __name__ == "__main__":
    fire.Fire(train_evaluate)

Overwriting training_app/train.py


### Package the script into a docker image.

Notice that we are installing specific versions of `scikit-learn` and `pandas` in the training image. This is done to make sure that the training runtime is aligned with the serving runtime. Later in the notebook you will deploy the model to AI Platform Prediction, using the 1.15 version of AI Platform Prediction runtime. 

Make sure to update the URI for the base image so that it points to your project's **Container Registry**.

In [68]:
%%writefile {TRAINING_APP_FOLDER}/Dockerfile

FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

Overwriting training_app/Dockerfile


### Build the docker image. 

You use **Cloud Build** to build the image and push it your project's **Container Registry**. As you use the remote cloud service to build the image, you don't need a local installation of Docker.

In [69]:
IMAGE_NAME='trainer_image'
IMAGE_TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, IMAGE_TAG)

In [70]:
!gcloud builds submit --tag $IMAGE_URI $TRAINING_APP_FOLDER

Creating temporary tarball archive of 3 file(s) totalling 4.2 KiB before compression.
Uploading tarball of [training_app] to [gs://assignment-371420_cloudbuild/source/1670970304.077718-7dc210cc06b3452095800ee818794aa7.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/assignment-371420/locations/global/builds/70fbeefa-9150-40ce-abca-4f8d81170db8].
Logs are available at [ https://console.cloud.google.com/cloud-build/builds/70fbeefa-9150-40ce-abca-4f8d81170db8?project=725195911767 ].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "70fbeefa-9150-40ce-abca-4f8d81170db8"

FETCHSOURCE
Fetching storage object: gs://assignment-371420_cloudbuild/source/1670970304.077718-7dc210cc06b3452095800ee818794aa7.tgz#1670970304390369
Copying gs://assignment-371420_cloudbuild/source/1670970304.077718-7dc210cc06b3452095800ee818794aa7.tgz#1670970304390369...
/ [1 files][  1.9 KiB/  1.9 KiB]                                                
Operation com

## Submit an AI Platform hyperparameter tuning job

### Create the hyperparameter configuration file. 
Recall that the training code uses `SGDClassifier`. The training application has been designed to accept two hyperparameters that control `SGDClassifier`:
- Max iterations
- Alpha

The below file configures AI Platform hypertuning to run up to 6 trials on up to three nodes and to choose from two discrete values of `max_iter` and the linear range betwee 0.00001 and 0.001 for `alpha`.

In [71]:
%%writefile {TRAINING_APP_FOLDER}/hptuning_config.yaml

# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#            http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

trainingInput:
  hyperparameters:
    goal: MAXIMIZE
    maxTrials: 4
    maxParallelTrials: 4
    hyperparameterMetricTag: accuracy
    enableTrialEarlyStopping: TRUE 
    params:
    - parameterName: max_iter
      type: DISCRETE
      discreteValues: [
          200,
          500
          ]
    - parameterName: alpha
      type: DOUBLE
      minValue:  0.00001
      maxValue:  0.001
      scaleType: UNIT_LINEAR_SCALE

Overwriting training_app/hptuning_config.yaml


### Start the hyperparameter tuning job.

Use the `gcloud` command to start the hyperparameter tuning job.

In [72]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
--config $TRAINING_APP_FOLDER/hptuning_config.yaml \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--hptune

Job [JOB_20221213_222801] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20221213_222801

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20221213_222801
jobId: JOB_20221213_222801
state: QUEUED


### Monitor the job.

You can monitor the job using Google Cloud console or from within the notebook using `gcloud` commands.

In [73]:
!gcloud ai-platform jobs describe $JOB_NAME

createTime: '2022-12-13T22:28:05Z'
etag: pFISyjSQYas=
jobId: JOB_20221213_222801
jobPosition: '0'
state: PREPARING
trainingInput:
  args:
  - --training_dataset_path=gs://artifacts.assignment-371420.appspot.com/data/training/dataset.csv
  - --validation_dataset_path=gs://artifacts.assignment-371420.appspot.com/data/validation/dataset.csv
  - --hptune
  hyperparameters:
    enableTrialEarlyStopping: true
    goal: MAXIMIZE
    hyperparameterMetricTag: accuracy
    maxParallelTrials: 4
    maxTrials: 4
    params:
    - discreteValues:
      - 200.0
      - 500.0
      parameterName: max_iter
      type: DISCRETE
    - maxValue: 0.001
      minValue: 1e-05
      parameterName: alpha
      scaleType: UNIT_LINEAR_SCALE
      type: DOUBLE
  jobDir: gs://artifacts.assignment-371420.appspot.com/jobs/JOB_20221213_222801
  masterConfig:
    imageUri: gcr.io/assignment-371420/trainer_image:latest
  region: us-central1
trainingOutput:
  isHyperparameterTuningJob: true

View job in the Cloud Conso

In [74]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2022-12-13 22:28:05 +0000	service		Validating job requirements...
INFO	2022-12-13 22:28:05 +0000	service		Job creation request has been successfully validated.
INFO	2022-12-13 22:28:05 +0000	service		Job JOB_20221213_222801 is queued.
INFO	2022-12-13 22:28:16 +0000	service	2	Waiting for job to be provisioned.
INFO	2022-12-13 22:28:16 +0000	service	1	Waiting for job to be provisioned.
INFO	2022-12-13 22:28:16 +0000	service	3	Waiting for job to be provisioned.
INFO	2022-12-13 22:28:16 +0000	service	4	Waiting for job to be provisioned.
INFO	2022-12-13 22:28:17 +0000	service	1	Waiting for training program to start.
INFO	2022-12-13 22:28:18 +0000	service	2	Waiting for training program to start.
INFO	2022-12-13 22:28:18 +0000	service	3	Waiting for training program to start.
INFO	2022-12-13 22:28:18 +0000	service	4	Waiting for training program to start.
NOTICE	2022-12-13 22:28:50 +0000	master-replica-0.gcsfuse	2	Opening GCS connection...
NOTICE	2022-12-13 22:28:50 +0000	master-replica-0.

**NOTE: The above AI platform job stream logs will take approximately 5~10 minutes to display.**

### Retrieve HP-tuning results.

After the job completes you can review the results using Google Cloud Console or programatically by calling the AI Platform Training REST end-point.

In [75]:
ml = discovery.build('ml', 'v1')

job_id = 'projects/{}/jobs/{}'.format(PROJECT_ID, JOB_NAME)
request = ml.projects().jobs().get(name=job_id)

try:
    response = request.execute()
except errors.HttpError as err:
    print(err)
except:
    print("Unexpected error")
    
response

{'jobId': 'JOB_20221213_222801',
 'trainingInput': {'args': ['--training_dataset_path=gs://artifacts.assignment-371420.appspot.com/data/training/dataset.csv',
   '--validation_dataset_path=gs://artifacts.assignment-371420.appspot.com/data/validation/dataset.csv',
   '--hptune'],
  'hyperparameters': {'goal': 'MAXIMIZE',
   'params': [{'parameterName': 'max_iter',
     'type': 'DISCRETE',
     'discreteValues': [200, 500]},
    {'parameterName': 'alpha',
     'minValue': 1e-05,
     'maxValue': 0.001,
     'type': 'DOUBLE',
     'scaleType': 'UNIT_LINEAR_SCALE'}],
   'maxTrials': 4,
   'maxParallelTrials': 4,
   'hyperparameterMetricTag': 'accuracy',
   'enableTrialEarlyStopping': True},
  'region': 'us-central1',
  'jobDir': 'gs://artifacts.assignment-371420.appspot.com/jobs/JOB_20221213_222801',
  'masterConfig': {'imageUri': 'gcr.io/assignment-371420/trainer_image:latest'}},
 'createTime': '2022-12-13T22:28:05Z',
 'startTime': '2022-12-13T22:28:10Z',
 'endTime': '2022-12-13T22:36:57Z

The returned run results are sorted by a value of the optimization metric. The best run is the first item on the returned list.

In [76]:
response['trainingOutput']['trials'][0]

{'trialId': '3',
 'hyperparameters': {'alpha': '0.00026808203072969749', 'max_iter': '200'},
 'finalMetric': {'trainingStep': '1', 'objectiveValue': 0.703881073530879},
 'startTime': '2022-12-13T22:28:16.898598734Z',
 'endTime': '2022-12-13T22:35:54Z',
 'state': 'SUCCEEDED'}

## Retrain the model with the best hyperparameters

You can now retrain the model using the best hyperparameters and using combined training and validation splits as a training dataset.

### Configure and run the training job

In [77]:
alpha = response['trainingOutput']['trials'][0]['hyperparameters']['alpha']
max_iter = response['trainingOutput']['trials'][0]['hyperparameters']['max_iter']

In [78]:
JOB_NAME = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
JOB_DIR = "{}/{}".format(JOB_DIR_ROOT, JOB_NAME)
SCALE_TIER = "BASIC"

!gcloud ai-platform jobs submit training $JOB_NAME \
--region=$REGION \
--job-dir=$JOB_DIR \
--master-image-uri=$IMAGE_URI \
--scale-tier=$SCALE_TIER \
-- \
--training_dataset_path=$TRAINING_FILE_PATH \
--validation_dataset_path=$VALIDATION_FILE_PATH \
--alpha=$alpha \
--max_iter=$max_iter \
--nohptune

Job [JOB_20221213_223853] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe JOB_20221213_223853

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs JOB_20221213_223853
jobId: JOB_20221213_223853
state: QUEUED


In [79]:
!gcloud ai-platform jobs stream-logs $JOB_NAME

INFO	2022-12-13 22:38:56 +0000	service		Validating job requirements...
INFO	2022-12-13 22:38:56 +0000	service		Job creation request has been successfully validated.
INFO	2022-12-13 22:38:56 +0000	service		Waiting for job to be provisioned.
INFO	2022-12-13 22:38:56 +0000	service		Job JOB_20221213_223853 is queued.
INFO	2022-12-13 22:38:58 +0000	service		Waiting for training program to start.
NOTICE	2022-12-13 22:39:32 +0000	master-replica-0.gcsfuse		Opening GCS connection...
NOTICE	2022-12-13 22:39:32 +0000	master-replica-0.gcsfuse		Mounting file system "gcsfuse"...
NOTICE	2022-12-13 22:39:32 +0000	master-replica-0.gcsfuse		File system has been successfully mounted.
INFO	2022-12-13 22:41:06 +0000	master-replica-0		Copying file://model.pkl [Content-Type=application/octet-stream]...
INFO	2022-12-13 22:41:06 +0000	master-replica-0		/ [0 files][    0.0 B/  6.2 KiB]                                                
INFO	2022-12-13 22:41:06 +0000	master-replica-0		/ [1 files][  6.2 KiB/  6.2 Ki

**NOTE: The above AI platform job stream logs will take approximately 5~10 minutes to display.**

### Examine the training output

The training script saved the trained model as the 'model.pkl' in the `JOB_DIR` folder on Cloud Storage.

In [80]:
!gsutil ls $JOB_DIR

gs://artifacts.assignment-371420.appspot.com/jobs/JOB_20221213_223853/model.pkl


## Deploy the model to AI Platform Prediction

### Create a model resource

Use the gcloud command to create a model with model_name in $REGION tagged with labels.

In [81]:
model_name = 'forest_cover_classifier'
labels = "task=classifier,domain=forestry"

!gcloud ai-platform models create  $model_name \
 --regions=$REGION \
 --labels=$labels

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.models.create) Resource in projects [assignment-371420] is the subject of a conflict: Field: model.name Error: A model with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A model with the same name already exists.
    field: model.name


### Create a model version

Use the gcloud command to create a version of the model.



In [82]:
model_version = 'v01'

!gcloud ai-platform versions create {model_version} \
 --model={model_name} \
 --origin=$JOB_DIR \
 --runtime-version=1.15 \
 --framework=scikit-learn \
 --python-version=3.7\
 --region global

Using endpoint [https://ml.googleapis.com/]
ERROR: (gcloud.ai-platform.versions.create) ALREADY_EXISTS: Field: version.name Error: A version with the same name already exists.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: A version with the same name already exists.
    field: version.name


### Serve predictions
#### Prepare the input file with JSON formated instances.

In [83]:
input_file = 'serving_instances.json'

with open(input_file, 'w') as f:
    for index, row in X_validation.head().iterrows():
        f.write(json.dumps(list(row.values)))
        f.write('\n')

In [84]:
!cat $input_file

[3020.0, 135.0, 0.0, 0.0, 0.0, 4265.0, 219.0, 238.0, 156.0, 1015.0, "Rawah", "C7101"]
[2926.0, 0.0, 0.0, 192.0, 7.0, 3930.0, 218.0, 238.0, 156.0, 6432.0, "Rawah", "C7102"]
[3139.0, 0.0, 0.0, 0.0, 0.0, 2171.0, 218.0, 238.0, 156.0, 2636.0, "Rawah", "C7202"]
[2907.0, 0.0, 0.0, 560.0, 35.0, 3027.0, 218.0, 238.0, 156.0, 2054.0, "Rawah", "C7745"]
[3000.0, 270.0, 0.0, 313.0, 48.0, 4228.0, 217.0, 238.0, 158.0, 2089.0, "Rawah", "C7745"]


#### Invoke the model

Use the gcloud command send the data in $input_file to your model deployed as a REST API.

In [85]:
!gcloud ai-platform predict \
--model $model_name \
--version $model_version \
--json-instances $input_file\
--region global

Using endpoint [https://ml.googleapis.com/]
[0, 1, 0, 1, 1]


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>